In [1]:
ignore_keys=['rp','r','jk','cov','corr','jk_err','jkerr','data','data0']
class cov_corr():
    def mean_calc(self,data=[],axis=1):
        return np.mean(data,axis)

    def var_calc(self,data=[],axis=1):
        return np.var(data,axis)

    def residual_calc(self,data=[],axis=1): #calculate residual that go into cov calculation, y-E(y)
        mean=self.mean_calc(data,axis)
        dim0=len(mean)
        dim1=np.shape(data)[axis]
        #print dim0,dim1,axis
# since mean is always 1-d array, [mean]*dim1 always give matrix [dim0 X dim1]...
#hence depending on dimension of data may need to transpose to get correct residuals
        try:
            res=data-np.transpose([mean]*dim1)
            print ("transpose")
        except:
            res=data-[mean]*dim1
        return res

    def cov_matrix2(self,data=[],axis=1,ddof=0):
        return np.cov(data,rowvar=axis)

    def cov_matrix(self,data=[],axis=1,ddof=0):
        #return np.cov(data,rowvar=axis,ddof=1) #ddof=0 for correct variance
        mean=self.mean_calc(data,axis)
        dim0=len(mean)
        dim1=np.shape(data)[axis]
        res=self.residual_calc(data=data,axis=axis)
        cov=np.zeros((dim0,dim0))
        if len(res[0])!=dim1:
            res=np.transpose(res)#residual has dimension of data..
                                #depending on axis value, may need to transpose for correct covariance
        for i in np.arange(dim0):
            for j in np.arange(dim0):
                cov[i][j]+=np.sum(res[i]*res[j])
        cov/=np.float64(dim1-ddof)
        return cov

    def cross_cov_matrix2(self,data1=[],data2=[],axis=1,ddof=0):
        cross_cov=np.cov(data1,data2,rowvar=axis)
        N1=len(data1[0])
        N2=len(data2[0])
        cov2=np.zeros((N1,N2))
        for i in np.arange(N1):
            for j in np.arange(N2):
                cov2[i][j]=cross_cov[i+N1][j]
        return cov2

    def cross_cov_matrix(self,data1=[],data2=[],axis=1,ddof=0):
    #return np.cov(data,rowvar=axis,ddof=1) #ddof=0 for correct variance
        mean1=self.mean_calc(data1,axis)
        mean2=self.mean_calc(data2,axis)
        dim0=len(mean1)
        dim2=len(mean2)
        dim1=np.shape(data1)[axis]
        res1=self.residual_calc(data=data1,axis=axis)
        res2=self.residual_calc(data=data2,axis=axis)
        cov=np.zeros((dim0,dim2))
        if dim0!=dim2:
            print ('cross cov: dimensions of data 1 and data2 not equal')
        if len(res1[0])!=dim1:
            res2=np.transpose(res2)
            res1=np.transpose(res1)#residual has dimension of data..
                            #depending on axis value, may need to transpose for correct covariance
        for i in np.arange(dim0):
            for j in np.arange(dim2):
                cov[i][j]+=np.sum(res1[i]*res2[j])
        cov/=np.float64(dim1-ddof)
        return cov

    #for some reason numpy covariance matrix has some constant multiplied to it..
    #seems to be some func of N but can't figure it out.
    #this func returns that constants value by comparing it with variance
    def var_cov_comparison(self,var=[],cov=[]):
        l=len(cov[0])
        v2=np.zeros_like(var)
        for i in np.arange(l):
            v2[i]=cov[i][i]
        r=var/v2
        x=r==r[0]
        if sum(x)<l:
            print ("variance-covariance miss match") #return full ratio array in case r is not constant
            return r  #note comparison can be easily fooled due to floating point error
        return [r[0]]

    def corr_matrix(self,cov_mat=[]): #correlation matrix
        diag=np.diag(cov_mat)
        return cov_mat/np.sqrt(np.outer(diag,diag))
        # dim=len(cov_mat[0])
        # var=np.zeros(dim)
        # corr=np.zeros((dim,dim))
        # for i in np.arange(dim):
        #     var[i]=cov_mat[i][i]
        # for i in np.arange(dim):
        #     for j in np.arange(dim):
        #         corr[i][j]=cov_mat[i][j]/np.sqrt(var[i]*var[j])
        # return corr

    def cross_corr_matrix(self,cov_mat=[],data1=[],data2=[],axis=1): #correlation matrix
        dim=len(cov_mat[0])
        var1=self.var_calc(data1,axis)
        var2=self.var_calc(data2,axis)
        corr=np.zeros((dim,dim))
#        for i in np.arange(dim):
 #           var[i]=cov_mat[i][i]
        for i in np.arange(dim):
            for j in np.arange(dim):
                corr[i][j]=cov_mat[i][j]/np.sqrt(var1[i]*var2[j])
        return corr

    #stack column from various dictionary elements to get a data matrix which goes into covariance calculations
    #incase dict elements have several columns, var defines the column to be used. better to have named columns
    def dict_stack(self,dic={},var='',ignore_keys=ignore_keys):
        keys=dic.keys()
        j=0
        mat=[]
        axis=0
        for i in keys:
            if i in ignore_keys:
                #print 'cov_corr.dict_stack::',i,'ignored in stacking'
                continue
            if j==0:
                if not check_empty(var):
                    mat=dic[i][var]
                else:
                    mat=dic[i]#[:]
                j=j+1
                continue
            if not check_empty(var):
                mat=np.vstack((mat,dic[i][var]))
            else:
                mat=np.vstack((mat,dic[i]))#dic[i][:]
        return mat,axis


    def dict_cov(self,dic={},var='',stack=0,axis=0,ddof=0,ignore_keys=ignore_keys):
        if stack!=0:
            #print "cov_corr: Stacking dictionary"
            mat,axis=self.dict_stack(dic=dic,var=var,ignore_keys=ignore_keys)
            #print "cov_corr: dictionary stacked"
        if stack==0:
            #print "no stacking:",var,axis
            mat=dic[var]
            axis=axis
        cov=self.cov_matrix2(data=mat,axis=axis,ddof=ddof)
        #print cov.shape
        corr=self.corr_matrix(cov)
        return cov,corr


    def dict_cross_cov(self,dic={},dic2={},var1='',var2='',stack=0,axis=0,ddof=0):
        if stack!=0:
            #print "cov_corr: Stacking dictionary"
            mat1,axis=self.dict_stack(dic=dic,var=var1)
            if dic2:
                mat2,axis=self.dict_stack(dic=dic2,var=var2)
            else:
                mat2,axis=self.dict_stack(dic=dic,var=var2)
        if stack==0:
            #print "no stacking:",var,axis
            mat1=dic[var1]
            if dic2:
                mat2=dic2[var2]
            else:
                mat2=dic[var2]
            axis=axis
        cov=self.cross_cov_matrix2(data1=mat1,data2=mat2,axis=axis,ddof=ddof)
        corr=self.cross_corr_matrix(cov_mat=cov,data1=mat1,data2=mat2,axis=axis)
        return cov,corr

    def matrix_diagonalize(self,m=[]):
        N1=len(m)
        N2=len(m[0])
        for i in np.arange(N1):
            for j in np.arange(N2):
                if i==j:
                    continue
                m[i][j]=0
        return m

    def mat_inv(self,m=[]):
        return np.linalg.inv(m)

    def matrix_cut(self,mat=[],x=[]):
        m=mat[x]
        N=sum(x)
        m2=np.zeros((N,N))
        j=0
        for i in m:
            m2[j]=i[x]
            j=j+1
        return m2

    def chi_sq(self,theory=[],data=[],x=[],cov=[]):
        if len(x):
            data=data[x]
            theory=theory[x]
            cov=self.matrix_cut(mat=cov,x=x)
        cov_inv=self.mat_inv(m=cov)
        #print data.shape,theory.shape,cov.shape,cov_inv.shape
        N=len(data)
        chi_sq=0
        for i in np.arange(N):
            for j in np.arange(N):
                chi_sq+=((data[i]-theory[i])*(data[j]-theory[j])*(cov_inv[i][j]))
        #chi_sq/=2.
        return chi_sq,N


In [2]:
def jk_read(dataset='',var=None,njk=100,auto_cross='_cross',bins='_bins',
            home='',corr='_final',skiprows=0,**kwargs):
    data_jk={}
    filename=dataset
    try:
        data_jk['data']=np.genfromtxt(home+filename+bins+auto_cross+'_jk_final.dat',
                                      names=True,skip_header=skiprows)
    except:
        print ('file not found',home+filename+bins+auto_cross+'_jk_final.dat')
    try:
        data_jk['data0']=np.genfromtxt(home+filename+bins+auto_cross+corr+'.dat',
                                       names=True,skip_header=skiprows)
    except:
        print ('file not found',home+filename+bins+auto_cross+corr+'.dat')
    for i in np.arange(njk):
        data_jk[i]=np.genfromtxt(home+filename+bins+auto_cross+'_jk'+str(i)+corr+'.dat',names=True,
                                 skip_header=skiprows)
    cc=cov_corr()
    if var and njk>0:
        data_jk['cov']={}
        data_jk['corr']={}
        for v in var:
            cov,corr=cc.dict_cov(dic=data_jk,var=v,stack=1,ignore_keys=['data','data0','cov','corr'])
            cov*=njk-1.
            data_jk['cov'][v]=cov
            data_jk['corr'][v]=corr
    return data_jk


In [3]:
file_home='./temp/hsc_test/corr_data_out/'
file_name='hsc_test'

In [4]:
njk=0

In [18]:
w_jk=jk_read(dataset=file_name,var=['wgg','wgp'],njk=njk,auto_cross='_cross',bins='_bins',
            home=file_home,corr='_final',skiprows=0)

file not found ./temp/hsc_test/corr_data_out/hsc_test_bins_cross_jk_final.dat


In [19]:
w_jk.keys()

dict_keys(['data0'])

In [20]:
w_jk['data0']

array([(0.129245,    3328., 0., 0.,  0.00934529,  0., -0.00680549, -0., 0., 0., 0., 0., 0., 0., -6.88021, -0., 0.,    3328., 0.1     ,  0.158489, 0.),
       (0.204839,    7185., 0., 0., -0.00258386, -0.,  0.00872609,  0., 0., 0., 0., 0., 0., 0., -6.86842, -0., 0.,    7185., 0.158489,  0.251189, 0.),
       (0.324648,   15355., 0., 0., -0.00316065, -0., -0.00174473, -0., 0., 0., 0., 0., 0., 0., -6.90942, -0., 0.,   15355., 0.251189,  0.398107, 0.),
       (0.514532,   31891., 0., 0., -0.00308426, -0.,  0.00314672,  0., 0., 0., 0., 0., 0., 0., -6.9076 , -0., 0.,   31891., 0.398107,  0.630957, 0.),
       (0.815479,   67748., 0., 0., -0.00120796, -0., -0.00215722, -0., 0., 0., 0., 0., 0., 0., -6.9079 , -0., 0.,   67748., 0.630957,  1.      , 0.),
       (1.29245 ,  132112., 0., 0.,  0.00112028,  0.,  0.00357882,  0., 0., 0., 0., 0., 0., 0., -6.90481, -0., 0.,  132112., 1.      ,  1.58489 , 0.),
       (2.04839 ,  255198., 0., 0.,  0.0012611 ,  0., -0.0005873 , -0., 0., 0., 0., 0., 0., 0.

In [21]:
w_jk['data0']['npairs']

array([   3328.,    7185.,   15355.,   31891.,   67748.,  132112.,
        255198.,  513393., 1002919., 1746213.])

In [22]:
w_jk['data0']['wgp']

array([ 0.00934529, -0.00258386, -0.00316065, -0.00308426, -0.00120796,
        0.00112028,  0.0012611 ,  0.00048746, -0.000622  , -0.00086452])

'data': contains the jackknife mean and the jackknife errorbars

'data0': Contains the full sample measurements. Do not use the errorbars from this set.

0....njk-1: Contains measurements from the respective jackknife regions.

cov, corr: Covariance and correlation matrices for the variables passed.

In [8]:
#following are for debugging if needed
RRjk=jk_read(dataset=file_name,var=['wgg'],njk=njk,auto_cross='_cross',bins='_bins',
            home=file_home,corr='_RR',skiprows=0)
RR2jk=jk_read(dataset=file_name,var=['wgg'],njk=njk,auto_cross='_cross',bins='_bins2D',
            home=file_home,corr='_RR',skiprows=0)
SDjk=jk_read(dataset=file_name,var=['wgg'],njk=njk,auto_cross='_cross',bins='_bins',
            home=file_home,corr='_SD',skiprows=0)
SD2jk=jk_read(dataset=file_name,var=['wgg'],njk=njk,auto_cross='_cross',bins='_bins2D',
            home=file_home,corr='_SD',skiprows=0)

SRjk=jk_read(dataset=file_name,var=['wgg'],njk=njk,auto_cross='_cross',bins='_bins',
            home=file_home,corr='_SR',skiprows=0)

file not found ./temp/hsc_test/corr_data_out/hsc_test_bins_cross_jk_final.dat
file not found ./temp/hsc_test/corr_data_out/hsc_test_bins2D_cross_jk_final.dat
file not found ./temp/hsc_test/corr_data_out/hsc_test_bins_cross_jk_final.dat
file not found ./temp/hsc_test/corr_data_out/hsc_test_bins2D_cross_jk_final.dat
file not found ./temp/hsc_test/corr_data_out/hsc_test_bins_cross_jk_final.dat


# For derived quantities such as bias, compute jk errors as follows:

In [9]:
# say we want to compute bias using wgg/wmm ratio
wgg=jk_read(dataset=file_name,var=['wgg','wgp'],njk=njk,auto_cross='_cross',bins='_bins',
            home=file_home,corr='_final',skiprows=0) 

wmm=jk_read(dataset=file_name,var=['wgg','wgp'],njk=njk,auto_cross='_cross',bins='_bins',
            home=file_home,corr='_final',skiprows=0)

file not found ./temp/hsc_test/corr_data_out/hsc_test_bins_cross_jk_final.dat
file not found ./temp/hsc_test/corr_data_out/hsc_test_bins_cross_jk_final.dat


In [17]:
bias={}
njk=njk
for i in np.arange(njk):
    bias[i]=np.sqrt(wgg[i]['wgg']/wmm[i]['wgg'])
cc=cov_corr()
bias['cov'],bias['corr']=cc.dict_cov(dic=bias,stack=1) #in this example values are non-sensicle because we used the same calculations for wgg and wmm.

/verafs/scratch/phy200040p/sukhdeep/miniconda3/envs/intel/lib/python3.7/site-packages/ipykernel_launcher.py:91: RuntimeWarning: invalid value encountered in true_divide
